<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
  - [Инструкции](#toc1_1_)    
  - [Регистры](#toc1_2_)    
  - [Регистры х86](#toc1_3_)    
  - [Инструкция копирования MOV](#toc1_4_)    
  - [Простые арифметические инструкции](#toc1_5_)    
  - [Беззнаковое умножение и деление](#toc1_6_)    
  - [Инструкции работы со стеком](#toc1_7_)    
  - [Метки](#toc1_8_)    
  - [Инструкции безусловного перехода](#toc1_9_)    
  - [Функция](#toc1_10_)    
  - [Флаговый регистр RFLAGS](#toc1_11_)    
  - [Инструкции условного перехода](#toc1_12_)    
  - [Инструкции сравнения](#toc1_13_)    
  - [ABI (Application Binary Interface)](#toc1_14_)    
  - [System V ABI. Соглашение о передаче аргументов в функцию.](#toc1_15_)    
  - [Прерывания](#toc1_16_)    
    - [Тело обработчика прерывания](#toc1_16_1_)    
    - [Таблица дескрипторов прерываний (IDT)](#toc1_16_2_)    
    - [Запрет прерываний](#toc1_16_3_)    
- [Загрузка ОС](#toc2_)    
  - [BIOS](#toc2_1_)    
  - [Real Mode](#toc2_2_)    
- [Пример кода загрузчика BIOS](#toc3_)    
- [Загрузка Muliboot](#toc4_)    
- [Полезности для ассемблера в Linux:](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

## <a id='toc1_1_'></a>[Инструкции](#toc0_)

    Копирования (из памяти в регистр и назад, из регистра в регистр, из памяти в память (редко))
    Арифметические
    Перехода (условного и безусловного)
    Прочие инструкции

## <a id='toc1_2_'></a>[Регистры](#toc0_)

Это очень быстрые ячейки памяти

    Регистры специального назначения (указатель команд, флаговый регистр, многие многие другие)
    Регистры общего назначения

## <a id='toc1_3_'></a>[Регистры х86](#toc0_)

    Указатель команд RIP
    Флаговый регистр RFLAGS
    Общего назначения:  
        указатель стека RSP
        указатель "базы" RBP
        RAX, RBX, RCX, RDX, RSI, RDI, R8-R15   

## <a id='toc1_4_'></a>[Инструкция копирования MOV](#toc0_)

    movq <src>, <dst>

        movq %RAX, %RBX
        movq (%RAX), %RBX       # разыменование, т.е. взять значение по адресу, указанному в регистре
        movq $42, %RAX          # значение 42
        movq 42, %RBX           # значение по адресу 42
        movq $value, %RBX       # имена для адресов, $ - непосредственно значение адреса (без разыменования)
        movq value, %RBX        # разыменование, берется значение по адресу       

## <a id='toc1_5_'></a>[Простые арифметические инструкции](#toc0_)

    addq <src>, <dst>
    
        addq %RAX, %RBX         # value - значение по адресу
        addq %RAX, value
        addq $42, %RAX
    
    subq <src>, <dst>

    incq <op>

        incq %RAX
    
    decq <op>

## <a id='toc1_6_'></a>[Беззнаковое умножение и деление](#toc0_)

    mulq <op>:          # второй аргумент всегда регистр RAX
        RAX = (<op> * RAX) mod 2**64    # младшие и...
        RDX = (<op> * RAX) / 2**64      # старшие биты результата

    divq <op>:          # второй аргумент всегда регистр RAX
        RDX = (RDX * 2**64 + RAX) mod <op>      # остаток и...
        RAX = (RDX * 2**64 + RAX) / <op>        # частное от деления

## <a id='toc1_7_'></a>[Инструкции работы со стеком](#toc0_)

Стек - это область памяти, на который указывает регистр RSP (на последнее значение)  
В х86 говорят что стек растет вниз (более старые значения хранятся по большим/высоким адресам)  

    pushq <src>         # уменьшает значение RSP на 8 (суффикс q) и сохраняет по полученому адресу <src>

        pushq $42
        pushq %RAX

    popq <dst>          # удаляет значение со стека (увеличивает RSP на 8) и сохраняет его в <dst>

        popq %RAX

    movq (%RSP), %RAX   # если не хотим удалять, а только прочитать вершину стека



## <a id='toc1_8_'></a>[Метки](#toc0_)

Это просто имя для некоторого адреса. Синтаксис: имя:

        .data
    value:              # метка
        .quad 42        # место в памяти, где хранится 8байтное значение 42

        .text
    add42:              # метка на начало кода
        movq %rdi, %rax
        addq value, %rax
        retq


## <a id='toc1_9_'></a>[Инструкции безусловного перехода](#toc0_)

Изменяют значение регистра RIP

    jmp <label>     # переход на метку
    call <label>    # инструкция вызова функции
    retq            # инструкция возврата из функции


## <a id='toc1_10_'></a>[Функция](#toc0_)

Можно вызвать, должна вернуть управление в то место, откуда была вызвана

    main:                       # функция main
        movq $42, %rsi          
        call add42              # вызывает функцию add42 (процессор запоминает на стеке адрес следующей инстуркции для возврата)
        retq                    # на стеке Return Addr этой Инструкции

    # поэтому стек должен содержать актуальное значение адреса возврата, функция не должна его "испортить"

## <a id='toc1_11_'></a>[Флаговый регистр RFLAGS](#toc0_)

Хранит флаги
    
    ZF - результат операции 0
    CF - произошло беззнаковое переполнение (carry), бит переноса
    OF - произошло знаковое переполнение (overflow)
    ...
    

## <a id='toc1_12_'></a>[Инструкции условного перехода](#toc0_)

    jcc <label> - переход, если условие сс истино
        jz, je - проверяет ZF = 1
        jne, jnz - проверяет ZF = 0

*) грубо говоря, если результат предыдущей операции (например subq $42, %rax) == 0, то jz он же je перекинет на метку

        jg - больше (знаковый вариант)
        jge - больше или равно (знаковый вариант)
        ja - больше (беззнаковый вариант)
        jae - больше или равно (беззнаковый вариант)

## <a id='toc1_13_'></a>[Инструкции сравнения](#toc0_)

Арифметические инструкции изменяют флаги, но также изменяют свои аргументы!  
Есть команды, которые выставляют флаги, но не изменяют аргументы (просто проверяют условия):

    cmpq <src>, <dst> - вычисляет разность как subq, но не изменяет <dst>, а только выставляет флаги

Пример:  

    max:                        # вычисляет максимум двух переданных аргументов (rdi, rsi)
        movq %rdi, %rax         # предположим rdi больше, ответ будет в rax
        cmpq %rsi, %rdi         # rdi - rsi
        ja rdi_gt               # rdi - rsi > 0
        movq %rsi, %rax         # если rsi больше, то в rax rsi, иначе - пропустить эту инструкцию
    rdi_gt:
        ret                     # на выход

## <a id='toc1_14_'></a>[ABI (Application Binary Interface)](#toc0_)

как в функцию передаются параметры;  
как функция возвращает значения;  
какие регистры функция должна сохранить, а какие может испортить;  
и многое другое.

Разные компиляторы используют различные ABI:  

например, Microsoft используют свой собственный ABI;  
Unix-like системы, зачастую, используют System V ABI.  

Мы будем использовать System V ABI

## <a id='toc1_15_'></a>[System V ABI. Соглашение о передаче аргументов в функцию.](#toc0_)

Size of each argument gets rounded up to eightbytes. Therefore the stack will always be eightbyte aligned.
Выделяется 8 классов аргументов (указатели, целые, флоаты), все что больше 8 байт - класс MEMORY.

Основные регистры (предохраняется или нет):
    
    rax                         - 1-й возвращающий регистр (нет)
    rbx                         - base pointer (да)
    rcx                         - 4th integer argument (нет)
    rdx                         - 3rd arg; 2nd return reg (нет)
    rsp                         - указатель на стек (да)
    rbp                         - callee-saved  register; optionally used as frame pointer (да)
    rsi                         - 2nd arg (нет)
    rdi                         - 1st arg (нет)
    r8                          - 5th arg (нет)
    r9                          - pass 6th argument to functions (нет)
    r10                         - temporary register (нет)
    r11                         - временный регистр (нет)
    r12-r15                     - callee-saved registers (да)
    k0-k7 mmx0-mmx7 xmm8-xmm15  - временные регистры (нет)

## <a id='toc1_16_'></a>[Прерывания](#toc0_)

Прерывание - это событие, которое заставляет процессор прервать текущую задачу и вызвать специальный обработчик.  
Обработчики прерываний - это часть ядра ОС.
Прерывания могут происходить асинхронно, поэтому обработчик прерывания ответственен за сохранение состояния прерванной задачи.  
При вызове обработчика прерывания ПРОЦЕССОР сохраняет на стеке (для возврата после обработки прерывания): 

    SS          RSP + 40    # (Stack Segment) – содержит начальный адрес сегмента стека
    RSP         RSP + 32    # указатель стека
    RFLAGS      RSP + 24    # флаговый региста
    CS          RSP + 16    # (Code Segment) содержит начальный адрес сегмента кода. Этот адрес плюс значение смещения в командном указателе (RIP) определяет адрес команды, которая должна быть выбрана для выполнения. 
    RIP         RSP + 8     # адрес возврата (указатель команд)
    Error Code  RSP + 0     # этого в стеке может и НЕ БЫТЬ, тогда начинается с RIP

Обработчик прерывания обычно заканчивается инструкцией iretq. Если прерывание положило код ошибки на стек, то его нужно удалить со стека перед вызовом iretq.

### <a id='toc1_16_1_'></a>[Тело обработчика прерывания](#toc0_)

Процессор сохраняет самые общие данные об исполняемом коде. Обработчик должен сохранить (как минимум) регистры общего назначения, чтобы не поломать вычисления.  
А потом восстановить их перед завершением обработки прерывания.  


### <a id='toc1_16_2_'></a>[Таблица дескрипторов прерываний (IDT)](#toc0_)

Указывает, каким прерываниям какие обработчики соответствуют. В архитектуре x86:

    специальный регистр IDTR хранит адрес этой таблицы;
    инструкции LIDT и SIDT позволяют записать/прочитать регистр IDTR

В каждой записи IDT в полях Offset содержится 64-битный адрес сообтветствующего обработчика прерываний.  
IDT может содержать максимум 256 записей. Первые 32 зарезервированы под стандартные нужды.  
Остальные - под прерывания от внешних устройств. Либо:  

- драйвер при инициализации устройства запрашивает у ОС свободные IDT, передает устройству, которое использует затем его для прерывания. 
- настройка контроллера прерываний. Если устройство не может напрямую обращаться к процессору, то этим занимается контроллер устройства.  

Задача контроллера - арбитраж обработки прерываний от устройств.  Примеры контроллеров:

- PIC (prog.interrupt controller) (Intel 8259)
- APIC (advanced PIC) (Local PIC + IO PIC) более свежий вариант

### <a id='toc1_16_3_'></a>[Запрет прерываний](#toc0_)

Требуется, когда происходит например обновление некоторой структуры данных, до завершения которого, обращение к такой структуре нужно запредить, т.к. данные будут не консистентны.  
Как:  

- если устройство поддерживает такую операцию - запретить устройству генерировать прерывания.
- запретить контроллеру транслировать прерывания от устройств процессору.
- запретить процессору реагировать на прерывания:

        x86 RFLAGS содержит флаг IF (interrupt flag)
        инструкция cli обнуляет этот флаг - запрет прерываний
        инструкция sti устанавливает флаг обратно



# <a id='toc2_'></a>[Загрузка ОС](#toc0_)

Первая инструкция: 
 
    - x86 обращается по адресу 0xFFFFFFF0;
    - отвечает ему материнская карта по протоколу:

        - BIOS (Basic Input/Output System) - наследство IBM PC; 
        - UEFI (Unified Extensible Firmware Interface).
## <a id='toc2_1_'></a>[BIOS](#toc0_)

    - BIOS ищет диск, с которого можно прочитать первые 512 байт (a. k. a. загрузочный сектор);
    - последние 2 байта сектора должны хранить числа 0x55 и 0xAA (соответствует инструкциям push bp и stosb, но это не важно, оно не исполняется, а используется именно как сигнатура); 
    - сектор загружается в память по физическому адресу 0x7c00;
    - BIOS передает управление по физическому адресу 0x7c00.

## <a id='toc2_2_'></a>[Real Mode](#toc0_)

В это время процессор работает в Real Mode (доступная память 1 Мб, линейная адресация, вся доступная память доступна любому процессу - наследие i8086). В отличие от Protected Mode, когда включается страничная организация памяти и процессы изолированы по памяти.
    
    - Логический адрес состоит из двух частей: 
        - 16-битного сегмента (SEG) и 16-битного смещения (OFF);
        - физический адрес получается по формуле (SEG∗16 + OFF) mod 2**20.
    - Сегмент хранится в одном из специальных регистров: CS,DS,SS,ES,FS,GS

*) про формулу физического адреса: умножение на степень двойки это сдвиг влево (16 = 2\**4 на 4 бита), по модулю 2**20 это 20 младших бит результата. Тогда физический адрес 0x7c00 можно представить 4096 различными комбинациями 16-битных SEG:OFF (32 байта составной адрес. 20 байт отбрасываем. 12 остаётся "в начале": 2^12 = 4096). Это еще один крупный недостаток реального режима.

Задача кода MBR загрузчика прочитать с диска код, непоместившийся в первые 510 байт. Оставшийся код может быть кодом ОС, а может быть кодом (вторичного) загрузчика, например, GRUB.


Регистры общего назначения 16-битные:

    SP- указатель стека;
    BP- указатель "базы";
    AX,BX,CX,DX,SI,DI.

По мере роста разрядности, росли и регистры: RAX 64bit включает в себя EAX 32bit включает в себя AX (AH+AL) 16bit

# <a id='toc3_'></a>[Пример кода загрузчика BIOS](#toc0_)

		.code16				; директивы компилятору i8086
		.text				; .text указывает as, что он должен ассемблировать следующие операторы в конец подсекции text с номером (0 по умолчанию)
		.global start			; .global делает символ видимым для ld.
	start:
		ljmp $0x0, $real_start		; версия jmp, позволяющая явно указывать сегмент (CS <- 0, IP <- $real_start (адрес)): нужно только чтобы инициализировать сегментную часть адреса первой инструкции 0x7c00
	real_start:
		movw $0, %ax				; movw это 16 бит
		movw %ax, %ds				; напрямую в ds ss писать нельзя, только через регистр, такое вот странное ограничение x86
		movw %ax, %ss				; теперь сегментные регистры CS DS SS инициализированы нулем

		movw $0x7c00, %sp			; на стеке должна быть ссылка на область памяти, куда можно писать, и где нет нужных данных
		addw $0x0400, %sp			; поэтому к стартовому адресу (точке входа) добавляется смещение 0х0400 байт (4*16*16=1024 байт). А чего не 0х0200 ?? Проверено, 0х0200 ОК

		movw $0xB800, %ax			; видеобуфер ожидает данные по адресу 0xb8000 в памяти. Сегментом будет 0xb800 (при умножении на 16 станет 0xb8000).
		movw %ax, %es				; опять напрямую в сегментный регистр нельзя, заносим через %ax
		movw $data, %si				; ссылка (адрес) секции data заносится в %si
		movw $0, %di				; смещение 0x0000 заносим в %di, теперь %es:%di представляет собой сегмент:смещение для видеобуфера
		movw size, %cx				; размер секции data заносится в %cx, теперь %ds:%si представляет собой сегмент(0):смещение($data) для надписи(?)
		call memcpy				; вызов функции копирования для подготовленных данных (из %ds:%si (данные) в %es:%di (видеобуфер) скопировать %cx байт)

	loop:
		jmp loop				; цикл, в который входит процессор после завершения загрузочных операций. Процессор не может находиться в статическом состояни, 
							; он постоянно будет пытаться обращаться к памяти и интерпретировать, то что получает как инструкции, поэтому нужен такой цикл, из которого 
							; он будет выходить по мере обработки прерываний.
	
	memcpy:					; из %ds:%si (данные) в %es:%di (видеобуфер) копирует в цикле поштучно %cx байт
		testw %cx, %cx			; команда testw выполняет логическое И между всеми битами двух операндов. Результат никуда не записывается, команда влияет только на флаги (можно cmpw $0, %cx, но test вроде быстрее?)
		jz out				; если нечего копировать - на выход (к метке loop)
	again:					; иначе - цикл копирования
		movb (%si), %ah			; movb (1 байт) из %ds:(%si) (при этом %ds инициализирован 0 поэтому можно опустить) в старший байт регистра ax
		movb %ah, %es:(%di)		; оттуда в %es:(%di) - сегмент указан явно, смещение разыменовывается, т.е. значение заносится в адрес, содержащийся по смещению (%di)
		incw %si			; +1 (w значит размер аргумента word)
		incw %di			; +1
		decw %cx			; -1
		jnz again			; если %cx != 0, то еще разок
	out:
		ret

	data:
		.asciz "H\017e\017l\017l\017o\017!\017"		; директива ассемблирует каждую строку (символ) в последовательные адреса памяти, при выводе интерпретируется в рамках VGA text mode, т.е.:
								; (тут формат LE) первый байт - код симовла ASCII, потом коды цвета шрифта (4 бита), цвета фона (3 бита), мигания (1 бит)
	size:
		.short . - data					; директива формирует 2-байтовое число (.word - 4-байтовый вариант) размер аргумента: разницы между текущей точкой и меткой data, т.е. размер секции data.


# <a id='toc4_'></a>[Загрузка Muliboot](#toc0_)

... пока превышает мои возможности разобраться как работает. В общих чертах:  

    * Multiboot Specification version 0.6.96 describes machine state at the moment
    * when this code gets control in the section 3.2 "Machine state". Short
    * summary:
    *   - CPU is in 32 bit protected mode with paging and interrupts disabled;
    *   - register EBX contains physical address of multiboot info structure
    *     which we need to get memory map, so we have to preserve that address;
    *   - if we are going to use stack we need to initialize it ourself.
    *
    * So we need to do the following things:
    *   - check that CPU support 64 bit mode (long mode), we skip this check for
    *     the sake of simplicity
    *   - setup and enable paging
    *   - switch to 64 bit long mode
    *   - call into C code since we don't want to write everything in Assembly.
    


# <a id='toc5_'></a>[Полезности для ассемблера в Linux:](#toc0_)

Ассемблер в C: флаги сс для компиляции кода ассемблера в объектный файл загрузчика -ffreestanding -m32 

    hexdump -C boot.bin 	# dump + ASCII
    objdump -Dslx boot.o 	# disasm object file
    ndisasm boot.bin	# disasm bin file (из стандартного пакета binutils nasm)
    r2 boot.bin		# debug bin file (стронний топ пакет реверс-разработки radare2)